# Stimulus playground
This script is a playground to inspect and manipulate stimuli for a binocular rivalry (BR) experiment<br>
In this experiment (as at May 29th) I aim to present objects that are associated with a color (e.g., banana - yellow, strawberry - red) in rivarous situations.<br>
1. Show the same stimulus either in the "correct" and in an "incorrect" color (a banana in yellow and blue)
2. Show two different stimuli that are associated with different colors but in the same color (a banana and a strawberry both in yellow)

Compare the onset and sustained dominance with rivaling gratings that follow a series of non-rivaling gratings that appear to be rotating (Denison et al., 2011, Attarha et al., 2015)

## Stimuli
In the following I will look at the stimuli used by Teichmann et al., 2020

In [1]:
# import libraries

# use the os
import os
import glob
# math and data structure
import numpy as np
import pandas as pd
# plotting
import matplotlib.pyplot as plt
# image processing
import cv2

In [2]:
# stimuli from Teichmann et al., 2020
# provided by her OSF site (https://osf.io/tcqjh/)
stim_dir = os.path.join('..','stimuli')
teichmann_stim_dir = os.path.join(stim_dir,'teichmann_2020_stimuli')
blue_stim_dir = os.path.join(stim_dir,'blue_stimuli')

# taken from original - added alpha channel
true_color_stim_dir = os.path.join(stim_dir,'true_color')
if not os.path.exists(true_color_stim_dir):
    os.mkdir(true_color_stim_dir)
# stimuli in LAB space inverted
inverted_lab_stim_dir = os.path.join(stim_dir,'inverted_lab')
if not os.path.exists(inverted_lab_stim_dir):
    os.mkdir(inverted_lab_stim_dir)

# masks
mask_dir = os.path.join(stim_dir,'masks')
if not os.path.exists(mask_dir):
    os.mkdir(mask_dir)
foreground_background_mask_dir = os.path.join(stim_dir,'foreground_background_masks')
if not os.path.exists(foreground_background_mask_dir):
    os.mkdir(foreground_background_mask_dir)
    
# where results of image analyses are saved
results_dir = os.path.join(stim_dir,'analysis')
if not os.path.exists(results_dir):
    os.mkdir(results_dir)
ab_dist_plot_dir = os.path.join(results_dir, 'ab_distance')
if not os.path.exists(ab_dist_plot_dir):
    os.mkdir(ab_dist_plot_dir)
hue_dist_plot_dir = os.path.join(results_dir, 'hue_distributions')
if not os.path.exists(hue_dist_plot_dir):
    os.mkdir(hue_dist_plot_dir)

### Note
Some of the stimuli taken from Teichmann were prepared with the following cell, but created "wholes" in the stimuli, because these had pure white pixels in the foreground.<br>
The "wholes" within the images were filled using GIMP and replaced the originals.<br>
The true originals that were replaced are now located in `original_exchanged`

In [3]:
def create_foreground_background_masks(stim_path,output_path):
    # read in the original image
    img = cv2.imread(stim_path)
    
    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image to isolate the non-white pixels
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    # Find the contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a binary mask for the largest contour
    mask = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(mask, [largest_contour], -1, (255, 255, 255), cv2.FILLED)

    # Create a binary mask with the pixels inside the contour set to white
    # and the pixels outside the contour set to black
    binary_mask = cv2.inRange(mask, 255, 255)
    
    # convert image from BGR to HLS space
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    
    # The contour also captures empty space surrounded by object boundaries
    # We therefore add a mask that includes all pixels with a luminance value of 255 -> white
    white_mask = hls_img[:,:,1]!=255
    big_mask = big_mask = np.logical_and(white_mask, binary_mask)
    big_mask_img = big_mask.astype(np.uint8)*255
    big_mask_img = cv2.cvtColor(big_mask_img, cv2.COLOR_GRAY2BGR)
    
    cv2.imwrite(output_path,big_mask_img)

In [4]:
# get a list of all stimuli in the directory
teichmann_full_path = glob.glob(os.path.join(teichmann_stim_dir,'*.png'))
teichmann_full_path.sort()

# extract the image name from the stimulus path
teichmann_stimuli = [os.path.basename(stim) for stim in teichmann_full_path]

for stim in teichmann_stimuli:
    stim_path = os.path.join(teichmann_stim_dir, stim)
    mask_output_dir = os.path.join(foreground_background_mask_dir, stim)
    
    create_foreground_background_masks(stim_path=stim_path,output_path=mask_output_dir)

# BLUE
Green, red, orange and yellow stimuli were taken from Teichmann 2020 and share many properties<br>
The blue stimuli however were taken from google searches and cannot be automatically separated into foreground an background as the other stimuli can<br>
Inspect each of the blue stimuli by it self and try to find good parameters to separate them using `opencv`

In [5]:
def create_blue_background_foreground_masks(R_channel,G_channel,B_channel, R_thresh, G_thresh, B_thresh):
    _, thres_R = cv2.threshold(R, R_thresh[0], R_thresh[1], cv2.THRESH_BINARY_INV)
    _, thres_G = cv2.threshold(G, G_thresh[0], G_thresh[1], cv2.THRESH_BINARY_INV)
    _, thres_B = cv2.threshold(B, R_thresh[0], R_thresh[1], cv2.THRESH_BINARY_INV)
    # Threshold the image to isolate the non-white pixels
    rgb_thres = cv2.merge((thres_R,thres_G,thres_B))
    gray_thres = cv2.cvtColor(rgb_thres, cv2.COLOR_RGB2GRAY)
    # # Find the contours in the thresholded image
    contours, _ = cv2.findContours(gray_thres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    mask = np.zeros(gray_thres.shape, dtype=np.uint8)
    cv2.drawContours(mask, [largest_contour], -1, (255, 255, 255), cv2.FILLED)

    # Create a binary mask with the pixels inside the contour set to white
    # and the pixels outside the contour set to black
    binary_mask = cv2.inRange(mask, 255, 255)
    
    binary_mask_img = binary_mask.astype(np.uint8)
    binary_mask_img = cv2.cvtColor(binary_mask_img, cv2.COLOR_GRAY2BGR)
    
    return binary_mask

In [6]:
# NIVEA
stim = 'blue_nivea.png'
path = os.path.join(blue_stim_dir,stim)
img = cv2.imread(path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# RGB values of background are 71,112,76 - even though in the original png they are transparent
print(img_rgb[0,0])
r_threshold = [70,72]
g_threshold = [111,113]
b_threshold = [75,77]
# create mask using these values to extract the background
R, G, B = cv2.split(img_rgb)
nivea_mask = create_blue_background_foreground_masks(R_channel=R, G_channel=G,B_channel=B,
                                                     R_thresh=r_threshold,G_thresh=g_threshold,B_thresh=b_threshold)
# make the mask an image with three channels either white or black
cv2.imwrite(os.path.join(foreground_background_mask_dir,stim), nivea_mask)

[ 71 112  76]


True

In [7]:
# POOL
stim = 'blue_pool.png'
path = os.path.join(blue_stim_dir,stim)
img = cv2.imread(path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# RGB values of background are 255,255,255 for white background squares
print(img_rgb[0,0])
# RGB values of background are 238,239,239 for gray background squares
print(img_rgb[50,0])
gray_pool = cv2.cvtColor(img_rgb,cv2.COLOR_RGB2GRAY)
R, G, B = cv2.split(img_rgb)
r_threshold = [237,255]
g_threshold = [238,255]
b_threshold = [238,255]
pool_mask = create_blue_background_foreground_masks(R_channel=R, G_channel=G,B_channel=B,
                                                    R_thresh=r_threshold,G_thresh=g_threshold,B_thresh=b_threshold)
cv2.imwrite(os.path.join(foreground_background_mask_dir,stim), pool_mask)

[255 255 255]
[238 239 239]


True

In [8]:
# SIGN
stim = 'blue_sign.png'
path = os.path.join(blue_stim_dir,stim)
img = cv2.imread(path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# RGB values of background are 0,0,0 
print(img_rgb[0,0])
# RGB values of the circle and figures in the sign are white
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
black_mask = gray_img != 0
# make the mask an image with three channels either white or black
sign_mask_img = black_mask.astype(np.uint8)*255
sign_mask_img = cv2.cvtColor(sign_mask_img, cv2.COLOR_GRAY2BGR)
cv2.imwrite(os.path.join(foreground_background_mask_dir,stim), sign_mask_img)

[0 0 0]


True

### Distribution of HUE in foreground
We want to extract the Hue distribution within the images in order to create refined masks that only cover the parts of the object in the diagnostic color

## Background transparent
Make the background of the stimuli transparent

In [9]:
# get a list of all stimuli in the directory
stimuli_full_path = glob.glob(os.path.join(teichmann_stim_dir,'*.png'))
stimuli_full_path.sort()

# extract the image name from the stimulus path
teichmann_stimuli = [os.path.basename(stim) for stim in stimuli_full_path]

# read out the original stimuli and remove the background
for stim in teichmann_stimuli:
    # output directories
    teichmann_input_dir = os.path.join(teichmann_stim_dir, stim)
    true_color_output_dir = os.path.join(true_color_stim_dir,stim)
    current_mask_dir = os.path.join(foreground_background_mask_dir, stim)
    
    # read in the original image
    img = cv2.imread(teichmann_input_dir)
    mask = cv2.imread(current_mask_dir)
    mask = mask[:,:,0].astype(bool)
    
    bgra_img = cv2.cvtColor(img,cv2.COLOR_BGR2BGRA)
    bgra_img[~mask] = np.array([255,255,255,0],dtype='uint8')
    
    cv2.imwrite(true_color_output_dir, bgra_img)

#### Blue stimuli
Perform the same on the blue stimuli that are not derived from Teichmann et al., 2020

In [10]:
blue_stimuli = [
    'blue_nivea.png',
    'blue_pool.png',
    'blue_sign.png'
]
blue_stim_dir = os.path.join(stim_dir, 'blue_stimuli')
# read out the original stimuli and remove the background
for stim in blue_stimuli:
    # output directories
    blue_input_dir = os.path.join(blue_stim_dir, stim)
    true_color_output_dir = os.path.join(true_color_stim_dir,stim)
    current_mask_dir = os.path.join(foreground_background_mask_dir, stim)
    
    # read in the original image
    img = cv2.imread(blue_input_dir)
    mask = cv2.imread(current_mask_dir)
    mask = mask[:,:,0].astype(bool)
    
    bgra_img = cv2.cvtColor(img,cv2.COLOR_BGR2BGRA)
    bgra_img[~mask] = np.array([255,255,255,0],dtype='uint8')
    
    cv2.imwrite(true_color_output_dir, bgra_img)

In [11]:
# get a list of all stimuli in the directory
stimuli_full_path = glob.glob(os.path.join(true_color_stim_dir,'*.png'))
stimuli_full_path.sort()

# extract the image name from the stimulus path
stimuli = [os.path.basename(stim) for stim in stimuli_full_path]

In [11]:
for stim in stimuli:
    # output directories
    true_color_input_dir = os.path.join(true_color_stim_dir, stim)
    current_mask_dir = os.path.join(foreground_background_mask_dir, stim)
    
    # read in the original image
    img = cv2.imread(true_color_input_dir)
    mask = cv2.imread(current_mask_dir)
    mask = mask[:,:,0].astype(bool)
    
    # convert image from BGR to HLS space
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    H,L,S = cv2.split(hls_img)
    # create a histogram of the hue values in the image for later mask creation
    fig = plt.figure()
    plt.hist(H[mask],bins=50)
    plt.savefig(os.path.join(hue_dist_plot_dir,stim))
    plt.close(fig=fig)

## Invert the color in the stimuli
This is not performed in the same step because some minor adjustments were made in the true color stimuli using GIMP

In [12]:
def invert_img(input_dir, mask_dir, refined_mask_dir=None):
    # read in the original image
    img = cv2.imread(input_dir)
    mask = cv2.imread(mask_dir)
    mask = mask[:,:,0].astype(bool)
    
    if refined_mask_dir is None:
        refined_mask_dir = mask_dir
    
    refined_mask = cv2.imread(refined_mask_dir)
    refined_mask = refined_mask[:,:,0].astype(bool)
    
    # convert into LAB and LUV space for inversion
    lab_img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    
    # Split the L*a*b* image into its channels
    L, a, b = cv2.split(lab_img)
    a_inv = a.copy()
    b_inv = b.copy()
    # Invert the a and b channels
    a_inv[refined_mask] = 255 - a[refined_mask]
    b_inv[refined_mask] = 255 - b[refined_mask]
    
    # Merge the inverted channels back into the L*a*b* image
    inverted_lab_img = cv2.merge((L, a_inv, b_inv))
    inverted_lab_bgr_img = cv2.cvtColor(inverted_lab_img, cv2.COLOR_LAB2BGR)
    
    # add alpha channel to inverted image by converting from BGR to BGRA space
    inverted_lab_bgra_img = cv2.cvtColor(inverted_lab_bgr_img, cv2.COLOR_BGR2BGRA)
    
    # make background (everything outside the mask) transparent (and white)
    inverted_lab_bgra_img[~mask] = np.array([255,255,255,0],dtype='uint8')
    
    return inverted_lab_bgra_img

In [13]:
for stim in stimuli:
    # output directories
    true_color_input_dir = os.path.join(true_color_stim_dir, stim)
    current_mask_dir = os.path.join(foreground_background_mask_dir, stim)
    inverted_lab_output_dir = os.path.join(inverted_lab_stim_dir, stim)
    
    inverted_lab_bgra_img = invert_img(input_dir=true_color_input_dir, 
                                       mask_dir=current_mask_dir)
    # save images
    cv2.imwrite(inverted_lab_output_dir, inverted_lab_bgra_img)

# Refined Masks
To this point we just separated the foreground from the background<br>
However in the foreground might be some parts that we want to exclude (e.g., the green leaves on the tomato or the yellow seeds on the strawberry)<br>
We do so, by getting the "typical" hue value of the image and exclude pixels that are significantly different

In [14]:
color_mask_dir = os.path.join(stim_dir, 'color_masks')
if not os.path.exists(color_mask_dir):
    os.mkdir(color_mask_dir)

In [15]:
def create_color_mask(path, lower_threshold=0, upper_threshold=180):
    '''This function loads an image and create a mask of all pixels that lie within the object in question, 
    are not pure white and lie within a hue range.
    
    Input 
    path: path to the image that needs to be masked
    lower_threshold: lower threshold of the hue
    upper_threshold: upper threshold of the hue
    
    Output: mask
    
    The function first creates a mask covering the object by drawing a contour around the largest non-white object 
    (getting rid of unwanted pixels in the periphery).
    Since some images contain multiple objects that enclose a white surface the function create a second mask of 
    ALL white pixels.
    
    In a last step the function creates a mask of all pixels that lie within a range of hue values.
    This way we avoid pixels making up the a structure of the object that has a different memory color
    (e.g., yellow seeds in a strawberry)'''
    
    # read in the original image
    img = cv2.imread(path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Threshold the image to isolate the non-white pixels
    _, thresh = cv2.threshold(gray, 240, 255, cv2.THRESH_BINARY_INV)

    # Find the contours in the thresholded image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a binary mask for the largest contour
    mask = np.zeros(gray.shape, dtype=np.uint8)
    cv2.drawContours(mask, [largest_contour], -1, (255, 255, 255), cv2.FILLED)

    # Create a binary mask with the pixels inside the contour set to white
    # and the pixels outside the contour set to black
    binary_mask = cv2.inRange(mask, 255, 255)

    # convert image from BGR to HLS space
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
    H,L,S = cv2.split(hls_img)

    # The contour also captures empty space surrounded by object boundaries
    # We therefore add a mask that includes all pixels with a luminance value of 255 -> white
    white_mask = L != 255

    # Create a binary mask for pixels within the hue range
    hue_mask = cv2.inRange(H, lower_threshold, upper_threshold)
    
    # combine all masks into one big mask
    new_mask = white_mask & binary_mask & hue_mask
    
    return new_mask

#### Create the masks for the objects
Visually inspecting the histogram plots in `../stimuli/analysis/hue_distributions`<br>
Select the value deemed suitable and create the mask

In [16]:
# read in the csv file containing upper and lower limits for the color hue threshold
color_threshold_df = pd.read_csv(os.path.join(stim_dir, 'masking_thresholds.csv'))
# iterate over dataframe and create new masks based on values
for idx, item in color_threshold_df.iterrows():
    path = os.path.join(true_color_stim_dir,item.Stimulus)
    new_mask = create_color_mask(path=path, lower_threshold=item.lower, upper_threshold=item.upper)
    # convert mask from bool to black and white
    new_mask = new_mask.astype(np.uint8)*255
    # make mask three dimensional (for the shine toolbox)
    new_mask = cv2.cvtColor(new_mask, cv2.COLOR_GRAY2BGR)
    cv2.imwrite(os.path.join(color_mask_dir,item.Stimulus), new_mask)

### BLUE
Green, red, orange and yellow stimuli were taken from Teichmann 2020 and share many properties<br>
The blue stimuli however were taken from google searches and cannot be automatically separated into foreground an background as the other stimuli can<br>
Inspect each of the blue stimuli by it self and try to find good parameters to separate them using `opencv`

In [17]:
def create_blue_masks(R_channel,G_channel,B_channel, R_thresh, G_thresh, B_thresh):
    _, thres_R = cv2.threshold(R, R_thresh[0], R_thresh[1], cv2.THRESH_BINARY_INV)
    _, thres_G = cv2.threshold(G, G_thresh[0], G_thresh[1], cv2.THRESH_BINARY_INV)
    _, thres_B = cv2.threshold(B, R_thresh[0], R_thresh[1], cv2.THRESH_BINARY_INV)
    # Threshold the image to isolate the non-white pixels
    rgb_thres = cv2.merge((thres_R,thres_G,thres_B))
    gray_thres = cv2.cvtColor(rgb_thres, cv2.COLOR_RGB2GRAY)
    # # Find the contours in the thresholded image
    contours, _ = cv2.findContours(gray_thres, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # # Find the contour with the largest area
    largest_contour = max(contours, key=cv2.contourArea)

    mask = np.zeros(gray_thres.shape, dtype=np.uint8)
    cv2.drawContours(mask, [largest_contour], -1, (255, 255, 255), cv2.FILLED)

    # Create a binary mask with the pixels inside the contour set to white
    # and the pixels outside the contour set to black
    binary_mask = cv2.inRange(mask, 255, 255)
    # convert image from BGR to HLS space
    hls_img = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)

    # The contour also captures empty space surrounded by object boundaries
    # We therefore add a mask that includes all pixels with a luminance value of 255 -> white
    white_mask = hls_img[:,:,1] != 255

    big_mask = np.logical_and(white_mask, binary_mask)
    return big_mask

In [18]:
# NIVEA
stim = 'blue_nivea.png'
path = os.path.join(true_color_stim_dir,stim)
img = cv2.imread(path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# RGB values of background are 71,112,76 - even though in the original png they are transparent
print(img_rgb[0,0])
r_threshold = [70,72]
g_threshold = [111,113]
b_threshold = [75,77]
# create mask using these values to extract the background
R, G, B = cv2.split(img_rgb)
nivea_mask = create_blue_masks(R_channel=R, G_channel=G,B_channel=B,
                               R_thresh=r_threshold,G_thresh=g_threshold,B_thresh=b_threshold)
# make the mask an image with three channels either white or black
nivea_mask_img = nivea_mask.astype(np.uint8)*255
nivea_mask_img = cv2.cvtColor(nivea_mask_img, cv2.COLOR_GRAY2BGR)
cv2.imwrite(os.path.join(color_mask_dir,stim), nivea_mask_img)

[255 255 255]


True

In [19]:
# POOL
stim = 'blue_pool.png'
path = os.path.join(true_color_stim_dir,stim)
img = cv2.imread(path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# RGB values of background are 255,255,255 for white background squares
print(img_rgb[0,0])
# RGB values of background are 238,239,239 for gray background squares
print(img_rgb[50,0])
R, G, B = cv2.split(img_rgb)
r_threshold = [237,255]
g_threshold = [238,255]
b_threshold = [238,255]
pool_mask = create_blue_masks(R_channel=R, G_channel=G,B_channel=B,
                              R_thresh=r_threshold,G_thresh=g_threshold,B_thresh=b_threshold)
# make the mask an image with three channels either white or black
pool_mask_img = pool_mask.astype(np.uint8)*255
pool_mask_img = cv2.cvtColor(pool_mask_img, cv2.COLOR_GRAY2BGR)
cv2.imwrite(os.path.join(color_mask_dir,stim), pool_mask_img)

[255 255 255]
[255 255 255]


True

In [20]:
# SIGN
stim = 'blue_sign.png'
path = os.path.join(true_color_stim_dir,stim)
img = cv2.imread(path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# RGB values of background are 0,0,0 
print(img_rgb[0,0])
# RGB values of the circle and figures in the sign are white
print(img_rgb[200,310])
# get the grayscaled image and remove all white and black pixels
gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
white_mask = gray_img != 255
black_mask = gray_img != 0
sign_mask = white_mask & black_mask
# make the mask an image with three channels either white or black
sign_mask_img = sign_mask.astype(np.uint8)*255
sign_mask_img = cv2.cvtColor(sign_mask_img, cv2.COLOR_GRAY2BGR)
cv2.imwrite(os.path.join(color_mask_dir,stim), sign_mask_img)

[255 255 255]
[255 255 255]


True

### Refined inverted images
In the previous steps we inverted the color of the whole object
Now we create inverted images with the refined masks

In [21]:
# stimuli in LAB space inverted
refined_inverted_lab_stim_dir = os.path.join(stim_dir,'inverted_lab_masked')
if not os.path.exists(refined_inverted_lab_stim_dir):
    os.mkdir(refined_inverted_lab_stim_dir)

for stim in stimuli:
    # output directories
    true_color_input_dir = os.path.join(true_color_stim_dir, stim)
    current_mask_dir = os.path.join(foreground_background_mask_dir, stim)
    current_color_mask_dir = os.path.join(color_mask_dir,stim)
    inverted_lab_output_dir = os.path.join(refined_inverted_lab_stim_dir, stim)
    
    inverted_lab_bgra_img = invert_img(input_dir=true_color_input_dir, 
                                       mask_dir=current_mask_dir,
                                       refined_mask_dir=current_color_mask_dir)
    # save images
    cv2.imwrite(inverted_lab_output_dir, inverted_lab_bgra_img)

# Representative colors of stimuli
For each stimulus get a representative pixel of the true color and invered color stimulus

In [22]:
# dictionary to store results in
rep_pixel_dict = {
    'stimuli': [],
    'true_R': [],
    'true_G': [],
    'true_B': [],
    'inv_R': [],
    'inv_G': [],
    'inv_B': [],
}

stimulus_selection = [
    'blue_nivea.png',
    'blue_pool.png',
    'blue_sign.png',
    'green_brokkoli_1.png',
    'green_frog_1.png',
    'green_lettuce_1.png',
    'orange_basketball.png',
    'orange_carrots.png',
    'orange_pumpkin.png',
    'red_fire_extinguisher_1.png',
    'red_strawberry.png',
    'red_tomato.png',
    'yellow_banana.png',
    'yellow_chicken.png',
    'yellow_corn.png'
]

for stim in stimulus_selection:
    rep_pixel_dict['stimuli'].append(stim)
    # get paths of stimuli
    true_path = os.path.join(true_color_stim_dir,stim)
    inv_path = os.path.join(inverted_lab_stim_dir,stim)
    # read in the original and inverted stimuli
    true_color_img = cv2.imread(true_path)
    inv_color_img = cv2.imread(inv_path)
    # convert the image from BRG (opencv default) to RGB (everybodies default)
    true_color_img = cv2.cvtColor(true_color_img, cv2.COLOR_BGR2RGB)
    inv_color_img = cv2.cvtColor(inv_color_img, cv2.COLOR_BGR2RGB)
    
    # read in the mask of the stimulus
    mask_path = os.path.join(mask_dir, stim)
    mask_img = cv2.imread(mask_path)
    # the mask is saved as an image with three channels with either 0 or 255 values -> convert to boolean mask
    mask_img = mask_img[:,:,0].astype(bool)
    
    # extract the pixels within the mask
    true_pixels = true_color_img[mask_img==1]
    inv_pixels = inv_color_img[mask_img==1]
    
    # get the representative pixel by searching for the pixel values appearing most often
    # true pixel
    # unique_colors, counts = np.unique(true_pixels,axis=0,return_counts=True)
    # most_frequent_indices = np.flip(np.argsort(counts))
    # # we need to convert the pixel from np.uint8 to int. Otherwise the transversion to the dataframe messes things up
    # most_frequent_true_color = unique_colors[most_frequent_indices[0]].astype(int)
    most_frequent_true_color = true_pixels.mean(axis=0)
    rep_pixel_dict['true_R'].append(most_frequent_true_color[0])
    rep_pixel_dict['true_G'].append(most_frequent_true_color[1])
    rep_pixel_dict['true_B'].append(most_frequent_true_color[2])
    
    # inverted pixel
    # unique_colors, counts = np.unique(inv_pixels,axis=0,return_counts=True)
    # most_frequent_indices = np.flip(np.argsort(counts))
    # # we need to convert the pixel from np.uint8 to int. Otherwise the transversion to the dataframe messes things up
    # most_frequent_inverted_color = unique_colors[most_frequent_indices[0]].astype(int)
    most_frequent_inverted_color = inv_pixels.mean(axis=0)
    rep_pixel_dict['inv_R'].append(most_frequent_inverted_color[0])
    rep_pixel_dict['inv_G'].append(most_frequent_inverted_color[1])
    rep_pixel_dict['inv_B'].append(most_frequent_inverted_color[2])
    
# convert the dictionary to a dataframe
pixel_df = pd.DataFrame(data=rep_pixel_dict, columns=rep_pixel_dict.keys())
pixel_df.to_csv(os.path.join(stim_dir,'representative_pixels.csv'))

#### an image of the colors for each hue value

In [23]:
all_colors = np.zeros((100,180,3),dtype='uint8')
all_colors[:,:,0] = np.linspace(0,180,180)
all_colors[:,:,1] = 100
all_colors[:,:,2] = 150

all_colors_rgb = cv2.cvtColor(all_colors, cv2.COLOR_HLS2RGB)

fig = plt.figure()
plt.imshow(all_colors_rgb)
plt.savefig(os.path.join(mask_dir,'hue_distribution.png'))
plt.close(fig=fig)

### Luminance sanity check
Changing the color in LAB space resulted in pixels not representable in RGB space.<br>
In turn the conversion back results in different colors with different luminances<br>
<br>
Check the actual differences of luminance

In [24]:
lum_diff_dir = os.path.join(results_dir, 'lum_diff')
if not os.path.exists(lum_diff_dir):
    os.mkdir(lum_diff_dir)
    
lum_diff_sum = []
lum_diff_percent = []

for stim in stimuli:
    true_path = os.path.join(true_color_stim_dir, stim)
    inv_path = os.path.join(inverted_lab_stim_dir, stim)
    mask_path = os.path.join(mask_dir, stim)
    mask_img = cv2.imread(mask_path)
    mask_img = mask_img[:,:,0].astype(bool)

    true_color_img = cv2.imread(true_path)
    true_lab = cv2.cvtColor(true_color_img, cv2.COLOR_BGR2LAB)
    true_L, a, b = cv2.split(true_lab)
    
    inv_color_img = cv2.imread(inv_path)
    inv_lab = cv2.cvtColor(inv_color_img, cv2.COLOR_BGR2LAB)
    inv_L, a, b = cv2.split(inv_lab)

    true_L_f = true_L.astype(np.float32)
    inv_L_f = inv_L.astype(np.float32)
    
    lum_diff = (true_L_f-inv_L_f)[mask_img==1]
    lum_diff_sum.append(lum_diff.sum())
    lum_diff_percent.append(sum(lum_diff!=0)/len(lum_diff))
    
    fig = plt.figure()
    plt.hist(lum_diff, bins=40)
    plt.savefig(os.path.join(lum_diff_dir,stim))
    plt.close(fig=fig)